## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [84]:
# Imports and Setup
import pandas as pd
import numpy as np
import os

RAW_DATA_DIR = 'data/00-raw/'
INT_DATA_DIR = 'data/01-interim/'
PROCESSED_DATA_DIR = 'data/02-processed/'

In [1]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [13]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') 

import get_data 

datafiles = [
    { 'url': 'https://zenodo.org/records/17239943/files/dataset_2022_hash.zip?download=1', 'filename':'university_dropout_2022.zip'} # Decompressed later using pd.read_csv
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')



Overall Download Progress: 100%|██████████| 1/1 [00:22<00:00, 22.03s/it]                  

Successfully downloaded: university_dropout_2.zip


### Dataset #1 

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"
3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


In [20]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


### University Student Dropout Dataset

The University Student Dropout dataset is organized as yearly CSV files named dataset_{year}.csv, with each row corresponding to a student-course enrollment for that academic year. Each file integrates data from four sources: students, programs, courses, and digital logs, and also groups variables into six thematic categories: context, admission pathways, socio-economic and demographic background, academic data, digital logs, and Wi-Fi access. Contextual attributes include anonymized identifiers for students, courses, academic programs, and campuses, as well as the academic year and group IDs, capturing where and how each student is enrolled. Admission pathway variables describe how the student entered the university, including year of enrollment, type of admission, entry exam grades (scaled to 10 or 14), and program selection preference. Socioeconomic and demographic variables capture parental education, student dedication to studies, and whether the student had to move provinces to attend university, providing insight into economic or social challenges that might affect retention.

Academic data is the most detailed category, including grades, credits enrolled and earned across multiple years, semester performance, adjustments for credit recognition, internships, activities, and overall progress toward degree completion. Metrics like cumulative GPA, credits passed per semester, and credit completion rates across previous years allow for longitudinal assessment of academic success and dropout risk. Digital logs track Learning Management System (LMS) site engagement monthly, including number of visits, events, assignment and test submissions, total minutes spent online, and usage of course resources. For 2021 and 2022, Wi-Fi access records provide an additional proxy for on-campus presence, recording the number of days each student accessed the university network per month. All variables are anonymized using hash codes, and numerical metrics such as grades are scaled (e.g., 0–10 or 0–14 for entry exams), while credit counts are in academic credit units. LMS and Wi-Fi activity metrics are counts of actions, logins, or days.

While these metrics provide valuable insights, several concerns about the dataset should be noted. It is drawn from a single Spanish technological university, which limits generalizability to other fields or institutions, particularly in humanities or social sciences. Early dropouts may be underrepresented, and some variables, like parental education, employment, student dedication, may be self-reported and incomplete. Engagement measures may also reflect infrastructure availability or device usage rather than actual participation. Finally, identifiers are anonymized, which may reduce the precision of longitudinal tracking, and the data does not include periods affected by the COVID-19 pandemic, meaning it may not capture disruptions caused by virtual or hybrid learning environments. Despite these limitations, the dataset provides a detailed framework for studying factors influencing student retention and academic success.


In [66]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE

# A - Load the Dataset (Just the 2022 Subset for now - it is quite large)
data2 = pd.read_csv(
    f'{RAW_DATA_DIR}university_dropout_2022.zip',
    sep=';',
    compression='zip' # Webpage download default as .zip
)

/tmp/ipykernel_804/936537943.py:4: DtypeWarning: Columns (48,57,64,65,164) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv(


Based on the description of the dataset from the source on Zemodo, the dataset has been thouroughly tidied up, which is demontrated below. 

In [67]:
# B - Tidiness

# Show that each row is a single observation by cross checking duplicates against the identifiers for student, course, and degree hashes
duplicates = data2.duplicated(subset=['dni_hash', 'asi_hash', 'anyo_ingreso'])
print("Number of duplicate rows:", duplicates.sum())


# Show that columns are aptly named
print('='*50)
print(data2.columns)
print('='*50)
print(data2.dtypes) # note that for now, dtypes are often objects because pandas interprets the comma usage in certain numbers as a string most likely

# Show a preview of what the data looks like, demonstrating that columns are properly named, there are no overlapping values, and columns are generally meaningful
print('='*50)
data2.head(10)

Number of duplicate rows: 0
Index(['dni_hash', 'tit_hash', 'asi_hash', 'anyo_ingreso', 'tipo_ingreso',
       'nota10_hash', 'nota14_hash', 'campus_hash', 'estudios_p_hash',
       'estudios_m_hash',
       ...
       'n_resource_days_2023_6', 'pft_events_2023_7', 'pft_days_logged_2023_7',
       'pft_visits_2023_7', 'pft_assignment_submissions_2023_7',
       'pft_test_submissions_2023_7', 'pft_total_minutes_2023_7',
       'n_wifi_days_2023_7', 'resource_events_2023_7',
       'n_resource_days_2023_7'],
      dtype='object', length=169)
dni_hash                       object
tit_hash                       object
asi_hash                       object
anyo_ingreso                   object
tipo_ingreso                   object
                                ...  
pft_test_submissions_2023_7    object
pft_total_minutes_2023_7       object
n_wifi_days_2023_7             object
resource_events_2023_7         object
n_resource_days_2023_7         object
Length: 169, dtype: object


,dni_hash,tit_hash,asi_hash,anyo_ingreso,tipo_ingreso,nota10_hash,nota14_hash,campus_hash,estudios_p_hash,estudios_m_hash,...,n_resource_days_2023_6,pft_events_2023_7,pft_days_logged_2023_7,pft_visits_2023_7,pft_assignment_submissions_2023_7,pft_test_submissions_2023_7,pft_total_minutes_2023_7,n_wifi_days_2023_7,resource_events_2023_7,n_resource_days_2023_7
0,319636fc9270,620c9c332101,4596fcf257c4,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,319636fc9270,620c9c332101,81f4b5a1d0a8,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,319636fc9270,620c9c332101,442fcac005ed,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,319636fc9270,620c9c332101,3dc87ab71825,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,319636fc9270,620c9c332101,677c622c0bfb,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,319636fc9270,620c9c332101,2344965e8b89,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,319636fc9270,620c9c332101,5f52e54c6a9c,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,319636fc9270,620c9c332101,8b8b029f1142,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,319636fc9270,620c9c332101,705d739be21c,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,319636fc9270,620c9c332101,696d9363dc5a,"2012,0",NAP,NaN,"9,456",e4f95d56d90df35e,F,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# C - Size of Dataset
print("Dataset shape (rows, columns):", data2.shape)
print("Number of observations of student-course-year (rows):", data2.shape[0])
print("Number of variables (columns):", data2.shape[1])

Dataset shape (rows, columns): (159173, 169)
Number of observations of student-course-year (rows): 159173
Number of variables (columns): 169


As also mentioned in the paper connected to this dataset, there is a high systematic relationship in the missingness of much of the data, as well as a large portion of columns that have a lot of missing data. This is demonstrated below.

In [69]:
# D - Missing Data Exploration
# Basic exploratory analysis on the missing data as porportions and counts
missing_counts = data2.isnull().sum()
missing_percent = (missing_counts / len(data2)) * 100

missing_df = pd.concat([missing_counts, missing_percent], axis=1)
missing_df.columns = ['missing_count', 'missing_pct']

missing_df = missing_df[missing_df['missing_count'] > 0].sort_values(by='missing_pct', ascending=False)
missing_df

,missing_count,missing_pct
pft_test_submissions_2023_7,159148,99.984294
pft_assignment_submissions_2023_7,158800,99.765664
es_retitulado,158630,99.658862
total1,157656,99.046949
es_adaptado,156916,98.582046
...,...,...
rendimiento_cuat_a,12207,7.669014
rendimiento_cuat_b,9051,5.686266
rendimiento_total,8819,5.540513
estudios_m_hash,918,0.576731


In [70]:
# D - Missing Data Exploration
# A deeper dive into why some data is missing in the way it is
# Let us take a look at the missing wifi monitoring usage by campus
data2.groupby('campus_hash')['n_wifi_days_2023_7'] \
     .apply(lambda x: x.isnull().mean()*100) \
     .sort_values(ascending=False)

campus_hash
1398b376fdcce25c    88.458559
297c138806bdb5dd    87.812500
9103a6c82e355433    85.704161
3ca0e4af1c44f084    84.429455
7b778e4c1d1f33c9    83.958427
0f01a84bff1b2bf4    82.044018
60f19cd67252161d    79.815005
85ff657216cc9b54    79.775281
1a9d786be0ff0bfe    79.341426
6781b441c78d2643    78.329399
48c6e3d042649ef6    77.824773
234001f5d5f1eca4    77.424844
f9418773503e50b6    77.080491
47cfe5eb8ada0e74    76.700434
79df3742da86cfd4    76.659119
40f5b57b09f073ed    76.653696
86348ea0bf50ebf0    75.927487
4e808094851fc2ea    75.469381
16a36e86f6fed5d4    75.291622
e984139bcc2c5043    75.257732
f32b702fba23083f    74.931880
5d9d4510699dac58    74.718222
911ac1b13dac6fe9    73.259053
ddf9288fd8062579    72.413793
0672d49fe5a7035e    72.110665
eb074cd8374ba297    71.810089
f2a369a3b17169d7    71.753555
52025890fa603dbc    70.521364
2f4c06aba0f9a393    70.130678
0448d563bf72277a    70.024096
8138689887e6817e    68.799798
c8361f9b468e68c8    65.359477
e4f95d56d90df35e    64.51633

This clearly demonstrates that some campuses such as `bc5d84bed7dee3e1` have a very comparitively low missing percentage (38%) of their students' wifi utilization, while others, like `1398b376fdcce25c` have a very high missing percentage, around 88%. This clearly shows a systematic disparity in certain campus's ability to report such data. While every inconsistency cannot be described due to the sheer size of this dataset, this small subsample shows how there is a large systematic reason for certain data being missing. This is explored in a little more depth in the accompanying paper, but on a theoretical basis, because this dataset was compiled from various databases and resources and then homogenized, inconsistencies are bound to show. Furthermore, certain courses may be more open to utilizing LMS tools or adopting digital platforms for their education, resulting in systematic missingness in the data.

#### Outlier Discovery

The accompanying paper describes that during the data anonymization of students, suspicious variables were dealt with to protect anonymity. For example, if a certain aggregation of variables could identify a student, this was deleted. Furthermore duplicate entries were deleted. Furthermore, a general check to ensure data types remained consistent and that value ranges for data was well within the expected distribution accross datasets was conducted. 

#### Data Cleaning

This dataset features a high rate of missingness. As such, the general rule for now that we chose to go with was to delete any columns with a high threshold of missingness. In this case, we chose to drop the columns with more than 90% overall missingness (this means dropping around 30 columns), as even if this data may be useful, the sheer proportion of missing data would make it less impactful. While agressive, this will help us narrow down our scope for our final project. A test also revealed that attempting a super agressive drop of all rows with any sort of missing data would cut the dataset to only 162 entries, so this is also not used. However, entries with all NA entries were deleted. Another notable feature is that the csv was ';' deliminated and utilized commas as decimals, which is quite typical of much of Europe. As such, numbers are cleaned into decimal format and converted to float/int. 

For specific column-based adjustments, a few rules were established to deal with missingness:

1) Leave identifying hashes alone
2) Leave demographic/enrollment data alone
3) Fill credit/coursework work columns as 0 for NA entries
4) Fill activity/practical work columns as 0 for NA entries
5) Fill LMS/Wifi/Digital Engagement columns as 0 for NA entries

This was done because demographics, enrollement data, and identifying hashes being empty are likely a result of truely missing data. However, the other categories can be attributed to simply the absence of the student doing said column. For example, no entry for credits enrolled for a specific semester and for a specific courses may just mean that the student didn't take that course. 


In [89]:
# F - Data Cleaning

data_cleaned_2 = data2.copy()

# Step 1: Drop columns with >90% missingness
threshold = 0.90
data_cleaned_2 = data_cleaned_2.loc[:, data_cleaned_2.isna().mean() < threshold].copy()

# Step 2: Drop rows that are all NA
data_cleaned_2 = data_cleaned_2.dropna(axis=0, how='all')

# Step 3: Convert comma-based numbers to floats
for col in data_cleaned_2.select_dtypes(include='object').columns:
    try:
        data_cleaned_2[col] = data_cleaned_2[col].str.replace(',', '.').astype(float)
    except:
        pass  # leave non-numeric columns as object

# Step 4: Fill NA with 0 for predetermined count/activity columns
fillna_cols = [col for col in data_cleaned_2.columns 
               if any(x in col for x in ['n_wifi_days', 'resource_events', 'n_resource_days', 
                                         'pft_', 'actividades', 'total1', 'cred_mat', 'cred_sup'])]

for col in fillna_cols:
    if pd.api.types.is_numeric_dtype(data_cleaned_2[col]):
        data_cleaned_2[col] = data_cleaned_2[col].fillna(0)

data_cleaned_2.head()

,dni_hash,tit_hash,asi_hash,anyo_ingreso,tipo_ingreso,nota10_hash,nota14_hash,campus_hash,estudios_p_hash,estudios_m_hash,...,resource_events_2023_5,n_resource_days_2023_5,pft_events_2023_6,pft_days_logged_2023_6,pft_visits_2023_6,pft_total_minutes_2023_6,n_wifi_days_2023_6,resource_events_2023_6,n_resource_days_2023_6,n_wifi_days_2023_7
0,319636fc9270,620c9c332101,4596fcf257c4,2012.0,NAP,NaN,9.456,e4f95d56d90df35e,F,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,319636fc9270,620c9c332101,81f4b5a1d0a8,2012.0,NAP,NaN,9.456,e4f95d56d90df35e,F,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,319636fc9270,620c9c332101,442fcac005ed,2012.0,NAP,NaN,9.456,e4f95d56d90df35e,F,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,319636fc9270,620c9c332101,3dc87ab71825,2012.0,NAP,NaN,9.456,e4f95d56d90df35e,F,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,319636fc9270,620c9c332101,677c622c0bfb,2012.0,NAP,NaN,9.456,e4f95d56d90df35e,F,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
print("Shape of cleaned dataframe:", data_cleaned_2.shape)

print("\nData types:")
print(data_cleaned_2.dtypes.value_counts())

processed_file_path = os.path.join(PROCESSED_DATA_DIR, 'university_dropout_cleaned_2022.csv')
data_cleaned_2.to_csv(processed_file_path, index=False, sep=';')

print(f"\nCleaned dataset saved to: {processed_file_path}")

Shape of cleaned dataframe: (159173, 135)

Data types:
float64    115
object      13
int64        7
Name: count, dtype: int64

Cleaned dataset saved to: data/02-processed/university_dropout_cleaned_2022.csv


## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them